In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

In [2]:
cuda2 = False # torch.cuda.is_available() can be used to check if a gpu is available - I just set it to False
batch_size2 = 128
log_interval2 = 10
epochs2 = 4 # 10

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda2 else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda2 else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=True, **kwargs)

In [3]:
# Define VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc1a = nn.Linear(400, 100)
        self.fc21 = nn.Linear(100, 2) # Latent space of 2D
        self.fc22 = nn.Linear(100, 2) # Latent space of 2D
        self.fc3 = nn.Linear(2, 100) # Latent space of 2D
        self.fc3a = nn.Linear(100, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc1a(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.fc3a(h3))
        return torch.sigmoid(self.fc4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        temp = self.decode(z).view(-1, 1, 28, 28)
        return temp, mu, logvar

In [4]:
# Define CVAE model
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()

        self.fc1 = nn.Conv2d(1, 32, kernel_size=(
            3, 3), stride=2, padding=1)  # nn.Linear(784, 400)
        self.fc1a = nn.Conv2d(32, 64, kernel_size=(
            3, 3), stride=2, padding=1)  # nn.Linear(400, 100)
        W, H = 7, 7
        self.fc21 = nn.Linear(64 * W * H, 2)  # Latent space of 2D
        self.fc22 = nn.Linear(64 * W * H, 2)  # Latent space of 2D
        self.fc3 = nn.Linear(2, 64 * W * H)  # Latent space of 2D
        self.fc3a = nn.ConvTranspose2d(
            64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.fc4 = nn.ConvTranspose2d(
            32, 1, kernel_size=3, stride=2, padding=1, output_padding=1)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc1a(h1))
        h2 = h2.view(-1, 7 * 7 * 64)
        return self.fc21(h2), self.fc22(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h3 = h3.view(-1, 64, 7, 7)
        h4 = F.relu(self.fc3a(h3))
        return torch.sigmoid(self.fc4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [5]:


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD  # -ELBO


def mse(x, y):
    loss_fun = torch.nn.MSELoss()
    return loss_fun(x, y)


def train(model, optimizer, train_loader, epoch):
    # so that everything has gradients and we can do backprop and so on...
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()  # "reset" gradients to 0 for text iteration
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()  # calc gradients
        train_loss += loss.item()
        optimizer.step()  # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(model, test_loader, epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():  # no_grad turns of gradients...
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


def model_train(model, optimizer, train_loader, test_loader, epochs):
    for epoch in range(1, epochs + 1):
        train(model, optimizer, train_loader, epoch)
        test(model, test_loader, epoch)
        with torch.no_grad():
            sample = torch.randn(64, 2).to(device)  # 20 -> 2
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png')


In [6]:
vae_model = VAE().to(device)
vae_optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)
# train some shit

cvae_model = CVAE().to(device)
cvae_optimizer = optim.Adam(cvae_model.parameters(), lr=1e-3)
# train more shit

In [7]:
model_train(vae_model, vae_optimizer, train_loader, test_loader, 4)

====> Epoch: 1 Average loss: 186.2416
====> Test set loss: 165.8579
====> Epoch: 2 Average loss: 162.0740
====> Test set loss: 158.0092
====> Epoch: 3 Average loss: 156.5293
====> Test set loss: 153.9628
====> Epoch: 4 Average loss: 153.0760
====> Test set loss: 151.4412


In [8]:
model_train(cvae_model, cvae_optimizer, train_loader, test_loader, 4)

====> Epoch: 1 Average loss: 211.7008
====> Test set loss: 177.7465
====> Epoch: 2 Average loss: 173.2548
====> Test set loss: 169.7860
====> Epoch: 3 Average loss: 167.7287
====> Test set loss: 165.7367
====> Epoch: 4 Average loss: 164.6295
====> Test set loss: 163.5258
